In [2]:
from IPython.core.display import display, HTML, Markdown
display(HTML("<style>.container { width:90% !important; }</style>"))

In [3]:
import sys
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time
import os
import datetime
import pandas as pd
pd.set_option('display.max_colwidth', -1)
import json


__author__ = 'Myles'

## Setup Information
# chromedriver download: http://chromedriver.chromium.org/downloads
# ===========> CHANGE THIS
chromedriver_local_path = "/path/to/chromedriver"
logon_webpage = 'https://auth.udacity.com/sign-in?next=https%3A%2F%2Fmentor-dashboard.udacity.com%2Freviews%2Foverview'

# email and password that you use for udacity
# ===========> CHANGE THIS
email = "email@email.com"
# ===========> CHANGE THIS
password = "password"

## Feedbacks page Information
feedbacks_webpage = 'https://review-api.udacity.com/api/v1/me/student_feedbacks.json?start_date=2018-07-14'
## Review page Information
review_webpage = 'https://review-api.udacity.com/api/v1/me/submissions/completed.json'


class MentorReviewStudentFeedbacks(object):
    '''setUp() = Access Udacity's Mentor Review Page
       downloadFeedbacks() = Download feedback information
       downloadReviews() = Download review information
    '''
       

    def __init__(self,chromedriver_local_path,logon_webpage,email,password,review_webpage,feedbacks_webpage):
        # Allow Notifications
        self.opt = Options()
        # Pass the argument 1 to allow and 2 to block
        self.opt.add_experimental_option("prefs", { 
        #     "profile.default_content_setting_values.media_stream_mic": 1, 
        #     "profile.default_content_setting_values.media_stream_camera": 1,
        #     "profile.default_content_setting_values.geolocation": 1, 
            "profile.default_content_setting_values.notifications": 1 
        })
        # Setup
        self.chromedriver_local_path = chromedriver_local_path
        self.logon_webpage = logon_webpage
        self.email = email
        self.password = password
        self.driver = webdriver.Chrome(self.chromedriver_local_path,chrome_options=self.opt)
        self.driver.set_window_position(0, 0)
        self.driver.set_window_size(1466, 1124)
        # Email/Text info
        self.review_webpage = review_webpage
        self.feedbacks_webpage = feedbacks_webpage

    def setUp(self):
        # quick sign-in
        self.driver.get(self.logon_webpage)
        time.sleep(1)
        self.email_elem = WebDriverWait(self.driver, 15).until(EC.presence_of_element_located((By.CSS_SELECTOR, "input[type='email']")))
        self.email_elem.send_keys(self.email)
        self.password_elem = WebDriverWait(self.driver, 15).until(EC.presence_of_element_located((By.CSS_SELECTOR, "input[type='password']")))
        self.password_elem.send_keys(self.password + Keys.ENTER)
        time.sleep(2)
        
    def downloadFeedbacks(self,download=True):
        time_date = datetime.datetime.now().strftime("%I_%M%p_on_%b_%d")
        # access review content page
        self.driver.get(self.feedbacks_webpage)
        content = self.driver.find_element_by_xpath("html").text
        feedback_json = json.loads(content)
        feedback = pd.DataFrame(feedback_json)
        feedback = pd.concat([feedback.drop(['project'], axis=1), feedback['project'].apply(pd.Series)], axis=1)
        feedback["updated_at"] = pd.to_datetime(feedback.updated_at)
        feedback["Date"] = feedback.updated_at.map(lambda x: '{}/{}/{} at {}'.format(x.month,x.day,x.year, x.hour))
        feedback.sort_values(by='updated_at', inplace=True, ascending=False)
        if download:
            feedback.to_csv("student_feedbacks_{}.csv".format(time_date), encoding='utf-8', index=False)
        return feedback
        
    def downloadReviews(self,download=True):
        time_date = datetime.datetime.now().strftime("%I_%M%p_on_%b_%d")
        # access review content page
        self.driver.get(self.review_webpage)
        content = self.driver.find_element_by_xpath("html").text
        review_json = json.loads(content)
        reviews = pd.DataFrame(review_json)
        reviews = pd.concat([reviews.drop(['project'], axis=1), reviews['project'].apply(pd.Series)], axis=1)
        reviews["completed_at"] = pd.to_datetime(reviews.completed_at)
        reviews["Date"] = reviews.completed_at.map(lambda x: '{}/{}/{} at {}'.format(x.month,x.day,x.year, x.hour))
        reviews.sort_values(by='completed_at', inplace=True, ascending=False)
        if download:
            reviews.to_csv("reviews_{}.csv".format(time_date), encoding='utf-8', index=False)
        return reviews

    def tearDown(self):
        self.driver.quit()
        
# create instance
discourse = MentorReviewStudentFeedbacks(chromedriver_local_path,logon_webpage,email,password,review_webpage,feedbacks_webpage)
# log on to dashboard
discourse.setUp()
# review and feedback dataframes (feedback contains ratings)
reviews_df = discourse.downloadReviews()
feedback_df = discourse.downloadFeedbacks()
# Quit Driver
discourse.tearDown()



In [5]:
class CreateReviewsWebpage(object):
    '''browseLocal() = Open created webpage in browser
       createTable() = create HTML table from a list of lists (no limit on either dimensions)
                       Links in table will open to a new page by default.
       createContents() = basic webpage, insert table
    '''
    def __init__(self,reviewList,fileName,title = "Feedback Page"):
        '''Required inputs: A list of lists, a filename, and a page name'''
        self.reviewList = reviewList
        self.fileName = fileName
        self.title = title
        
    def createTable(self, listOfLists):
        """Create the HTML table, from the list of lists
        generated from the pandas dataframe (or other methods)"""
        htmlTable = "<table id='pyTable'><tr>"
        for ind, rows in enumerate(listOfLists):
            for data in rows:
                # add either the table headers (list 0) or the table rows (all other list items)
                if ind == 0:
                    htmlTable += "<th>{}</th>".format(data)
                else:
                    htmlTable += "<td>{}</td>".format(data)
            
            htmlTable += "</tr><tr>".format(data)
        # remove the last <tr>, close the table
        htmlTable = htmlTable[:-4] + "</table>"
        return htmlTable
    
    def createContents(self):
        '''Create the HTML page. The text will be the HTML table
        generated from createTable and the pandas data'''
        createdTable = self.createTable(self.reviewList)
        contents = """<!DOCTYPE html PUBLIC '-//W3C//DTD HTML 4.01 Transitional//EN'>
            <html>
            <head>
              <meta content='text/html; charset=ISO-8859-1'
             http-equiv='content-type'>
              <title>{}</title>
            </head>
            <style>
            #pyTable {{
                font-family: 'Trebuchet MS', Arial, Helvetica, sans-serif;
                border-collapse: collapse;
                width: 100%;
            }}
            #pyTable td, #pyTable th {{
                border: 1px solid #ddd;
                padding: 8px;
            }}
            #pyTable tr:nth-child(even){{background-color: #f2f2f2;}}
            #pyTable tr:hover {{background-color: #ddd;}}
            #pyTable th {{
                padding-top: 12px;
                padding-bottom: 12px;
                text-align: left;
                background-color: #4CAF50;
                color: white;
            }}
            </style>
            <body>{}
            </body>
            </html>""".format(self.title, createdTable)
        return contents

    def strToFile(self):
        """Create the HTML page. Then write the HTML file
        with the given name and the given contents."""
        self.contents = self.createContents()
        output = open(self.fileName,"w")
        output.write(self.contents)
        output.close()

    def browseLocal(self):
        '''Open chrome webbrowser with the local file created above
        with the given filename.'''
        import webbrowser, os.path
        self.strToFile()
        # Note: remove .get(using='chrome') if you are using a different browser to log on to udacity
        # removing it will open your system's default browser
        webbrowser.get(using='chrome').open("file:///" + os.path.abspath(self.fileName),new=2) #elaborated for Mac
        


In [6]:
# column options are: 'body', 'created_at', 'id', 'rating', 'read_at', 'submission_id', 'updated_at', 'id', 'name', 'visible', 'is_cert_project', 'is_career', 'Date', 'link'
includeCols = ['Date','rating','name','link','body']
# Optional Headers (otherwise column names are used). Has to have same len as includeCols
optHeaders = ['Date','Rating','Project','Link','Comment']

# choose columns to sort by
sortBy = ['updated_at']
# choose ordering. Has to have same len as sortBy
sortOrder = [False]

# choose HTML name
fileName = '{}_ratings_page.html'

#  All arguments set as defaults (change values above)
def callCreateReviewsWebpage(sortBy=sortBy, sortOrder = sortOrder , includeCols = includeCols, defaultHeaders = False, optHeaders = optHeaders, fileName=fileName):
    ## Sort by project name, then date (change to the option that you want):
    feedback_df.sort_values(by=sortBy, inplace=True, ascending=sortOrder)

    ## Create link (from submission number)
    feedback_df['link'] = feedback_df['submission_id'].apply(lambda x: f"<a href=https://review.udacity.com/#!/reviews/{x} target='_blank'>{x}</a>")

    ## Columns to include in HTML table
    dfHTML = feedback_df[includeCols].copy()
    
    ## Create separate pages for each project
    for ind, grp in dfHTML.groupby('name'):

        ## create a list of lists of entries
        myList = grp.values.tolist()    

        ## add the column headers (will become table headers)
        if defaultHeaders:
            myList.insert(0,list(grp.columns))
        else:
            ## or add your own headers
            myList.insert(0,optHeaders)


        reviewWebpage = CreateReviewsWebpage(myList,fileName=fileName.format(ind))
        reviewWebpage.browseLocal()
    
callCreateReviewsWebpage()